# Calulate Radiative Transfer Model

This notebook will load in a dataset and calculate a radiative transfer model to convert from radiance to reflectance.

This notebook pairs with the `process-to-reflectance` notebook.

The following is taken from and refers to the openhsi docs, specifically [loading and processing datacubes further](https://openhsi.github.io/openhsi/capture.html#Loading-and-processing-datacubes-further) and [getting surface reflectance](https://openhsi.github.io/openhsi/tutorial_camera.html#Getting-surface-reflectance)

Useful information on 6S can be found [here](https://py6s.readthedocs.io/en/latest/helpers.html#importing-atmospheric-profiles-from-radiosonde-data)

A couple of issues and discussion threads that assissted me:
- [rad_fit missing from data](https://github.com/openhsi/openhsi/issues/43)
- [surface reflectance discussion](https://robonationforum.vbulletin.net/forum/robotx/-2022-robotx-challenge/4206-getting-surface-reflectance)


In [1]:
from datetime import datetime
from openhsi.atmos import *
from openhsi.capture import *
from Py6S import *
from scipy.interpolate import interp1d
import numpy as np
import netCDF4 as nc
import pickle as pk

#### Processing levels

The library comes with some predefined recipes you can use to output a datacube with the desired level of processing. Depending on your use case, you may want to use save raw data, or choose a faster binning scheme. The available options are listed below.

| processing_lvl | Description |
|----------------|-------------|
-1 | do not apply any transforms (default)
0 | raw digital numbers cropped to useable sensor area
1 | crop + fast smile
2 | crop + fast smile + fast binning
3 | crop + fast smile + slow binning
4 | crop + fast smile + fast binning + conversion to radiance in units of uW/cm^2/sr/nm
5 | crop + fast smile + radiance + fast binning
6 | crop + fast smile + fast binning + radiance + reflectance
7 | crop + fast smile + radiance + slow binning
8 | crop + fast smile + radiance + slow binning + reflectance

Main difference between these is the order the transforms are used in the pipeline. This summaries the binning procedure and output:

| processing_lvl | Binning | Output |
|----------------|---------|--------|
-1,0,1 | None | Digital Numbers
2 | Fast | Digital Numbers
3 | Slow | Digital Numbers
4,5 | Fast | Radiance (uW/cm^2/sr/nm)
6 | Fast | Reflectance
7 | Slow | Radiance (uW/cm^2/sr/nm)
8 | Slow | Reflectance

## Setup and reading data

In [2]:
# path to the recorded image
data_path = "data/captured-data-bedford-1/2023_03_14/2023_03_14-01_26_05.nc"

# original camera settings
json_path = "assets/OpenHSI-11_settings_Mono12_bin2.json"
pkl_path  = "assets/OpenHSI-11_calibration_Mono12_bin2_window.pkl"

# processing level to use
proc_lvl = 4 # level to process the raw data to

# place to save processed datacube
raw_processed_path = "processed-data/bedford-capture"
processed_processed_path = raw_processed_path + "/reflectance"

# location/files to store updated camera settings in
updated_cam_settings_path = raw_processed_path + "/settings.json"
updated_cam_calibration_path = raw_processed_path + "/calibration.pkl"

In [3]:
# check data before reprocessing
data = nc.Dataset(data_path)
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): wavelength(528), x(456), y(650), time(650), temperature(650)
    variables(dimensions): int32 datacube(wavelength, x, y), int64 wavelength(wavelength), int64 x(x), int64 y(y), int64 time(time), float32 temperature(temperature)
    groups: 


In [4]:
# read unprocessed data into openhsi
proc_dc = ProcessRawDatacube(fname = data_path, processing_lvl=proc_lvl,
                             json_path=json_path, pkl_path=pkl_path)

Allocated 597.00 MB of RAM.
Allocated 142.60 MB of RAM.



Approximate location of data capture [here](https://www.google.com.au/maps/place/35%C2%B000'28.4%22S+138%C2%B034'17.2%22E/@-35.0083538,138.5718056,17.95z/data=!4m5!3m4!1s0x0:0xfb8cf0933aeb3896!8m2!3d-35.0078787!4d138.5715478)

radiosonde data available [here](http://weather.uwyo.edu/upperair/sounding.html)

### The following only needs to be run once for each data capture event

#### Initial data capture

In [5]:
# location at which the image was captured (can get this off of google maps)
proc_dc.settings["latitude"] = 35
proc_dc.settings["longitude"] = 138

# radiosonde settings
proc_dc.settings["radiosonde_region"] = "pac" # south pacific
proc_dc.settings["radiosonde_station_num"] = 94672 # adelaide airport (closest to us it seems)

# altitude at which the image was captured
proc_dc.settings["altitude"] = 1.5 # took images 1.5m off the ground, not sure if this is supposed to be sea level though.
# the docs say viewing zenith and azimuth, for solar go here https://gml.noaa.gov/grad/solcalc/
proc_dc.settings["zenith"] = 0. # directly above
proc_dc.settings["azimuth"] = 0. # directly above

# path to the 6S binary on the system running this
proc_dc.settings["sixs_path"] = "sixs"

#### Secondary data capture

In [6]:
# location at which the image was captured (can get this off of google maps)
proc_dc.settings["latitude"] = 35
proc_dc.settings["longitude"] = 138

# radiosonde settings
proc_dc.settings["radiosonde_region"] = "pac" # south pacific
proc_dc.settings["radiosonde_station_num"] = 94672 # adelaide airport (closest to us it seems)

# altitude at which the image was captured
proc_dc.settings["altitude"] = 2 # took images 1.5m off the ground, not sure if this is supposed to be sea level though.
# the docs say viewing zenith and azimuth, for solar go here https://gml.noaa.gov/grad/solcalc/
proc_dc.settings["zenith"] = 0. # directly above
proc_dc.settings["azimuth"] = 0. # directly above

# path to the 6S binary on the system running this
proc_dc.settings["sixs_path"] = "sixs"

#### Tertiary data capture

In [7]:
# location at which the image was captured (can get this off of google maps)
proc_dc.settings["latitude"] = 35
proc_dc.settings["longitude"] = 138

# radiosonde settings
proc_dc.settings["radiosonde_region"] = "pac" # south pacific
proc_dc.settings["radiosonde_station_num"] = 94672 # adelaide airport (closest to us it seems)

# altitude at which the image was captured
proc_dc.settings["altitude"] = 1.5 # took images 1.5m off the ground, not sure if this is supposed to be sea level though.
# the docs say viewing zenith and azimuth, for solar go here https://gml.noaa.gov/grad/solcalc/
proc_dc.settings["zenith"] = 0. # directly above
proc_dc.settings["azimuth"] = 0. # directly above

# path to the 6S binary on the system running this
proc_dc.settings["sixs_path"] = "sixs"

#### Sardi data capture

In [8]:
# location at which the image was captured (can get this off of google maps)
proc_dc.settings["latitude"] = -34.95233
proc_dc.settings["longitude"] = 138.50472

# radiosonde settings
proc_dc.settings["radiosonde_region"] = "pac" # south pacific
proc_dc.settings["radiosonde_station_num"] = 94672 # adelaide airport (closest to us it seems)

# altitude at which the image was captured
proc_dc.settings["altitude"] = 1.5 # took images 1.5m off the ground, not sure if this is supposed to be sea level though.
# the docs say viewing zenith and azimuth, for solar go here https://gml.noaa.gov/grad/solcalc/
proc_dc.settings["zenith"] = 0. # directly above
proc_dc.settings["azimuth"] = 0. # directly above

# path to the 6S binary on the system running this
proc_dc.settings["sixs_path"] = "sixs"

#### bedford data capture

In [9]:
# location at which the image was captured (can get this off of google maps)
proc_dc.settings["latitude"] = -35.02699
proc_dc.settings["longitude"] = 138.56960

# radiosonde settings
proc_dc.settings["radiosonde_region"] = "pac" # south pacific
proc_dc.settings["radiosonde_station_num"] = 94672 # adelaide airport (closest to us it seems)

# altitude at which the image was captured
proc_dc.settings["altitude"] = 1.5 # took images 1.5m off the ground, not sure if this is supposed to be sea level though.
# the docs say viewing zenith and azimuth, for solar go here https://gml.noaa.gov/grad/solcalc/
proc_dc.settings["zenith"] = 0. # directly above
proc_dc.settings["azimuth"] = 0. # directly above

# path to the 6S binary on the system running this
proc_dc.settings["sixs_path"] = "sixs"

#### This will take some time but again, only needs to be complete once for each data capture event

In [10]:
# build model
model = Model6SV(lat = proc_dc.settings["latitude"], lon = proc_dc.settings["longitude"],
                 z_time = datetime.strptime(proc_dc.settings["datetime_str"],"%Y-%m-%d %H:%M"),
                 station_num = proc_dc.settings["radiosonde_station_num"], region = proc_dc.settings["radiosonde_region"],
                 alt = proc_dc.settings["altitude"], zen = proc_dc.settings["zenith"], azi = proc_dc.settings["azimuth"], # viewing zenith and azimuth angles
                 aero_profile = AeroProfile.Maritime,
                 wavelength_array = np.linspace(350,900,num=2000), # choose larger range than sensor range
                 sixs_path = proc_dc.settings["sixs_path"])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [11:46<00:00,  2.83it/s]


In [13]:
# add the model to the datacube for processing
proc_dc.calibration["rad_fit"] = interp1d(np.linspace(350,900,num=2000), model.radiance/10, kind='cubic')

In [14]:
# save the updated calibration files for future use
proc_dc.dump(updated_cam_settings_path, updated_cam_calibration_path)